# Update SSO-created teacher account

As ArcGIS Online SSO assigns all new accounts to one role, this script can find certain  differences in usernames and then changes those users' roles. The use case that drove this script requested inbound teachers be promoted to publishers while students remained as users in the system.  Optionally, this script could be save to a .py file and set as a scheduled task.

*****

This script returns all the members of an organization and then filters out:
1. non-SSO usernames (assumes SSO usernames must end with underscore subdomain)
2. usernames older than 1 week 
3. usernames that don't contain six consecutive digits. This is a condition in this specific use case that differentiates student usernames from teacher usernames. You will need to change the regular expression used to differentiate between the teacher and student accounts in your situation.

A teacher tag is added and qualifying username promoted to org_publisher.

YOU SHOULD UPDATE LINES: 6, 12, 20(use your own regex)

Script based on ArcGIS Online API for Python 1.5.1, Python 3.6, Anconda 4.4

-trbaker/November 2018 

In [ ]:
# import libs and set vars
from arcgis.gis import GIS
import re
import time

gis = GIS("http://<your_org>.maps.arcgis.com", "<username>", "<password>", verify_cert=False)
gis_users = gis.users.search(query="", max_users=5000, outside_org=0)
newtags = ['teacher']

for userB in gis_users:
    # filter for SSO extension automatically added to usernames (e.g. janeSmith_KansasUniv)
    if userB.username.find('_<your_org>') >=0: 
        now = time.time()
        createSec=userB.created/1000  # var holds lastlogin in seconds (rather than milliseconds)
        diffCreateSec = now-createSec # var holds difference between now and lastlogin in seconds.
        diffCreateDays = diffCreateSec/(24*60*60)    #Last login in days (rounded)
        # filter for  new users from the past week.
        if round(diffCreateDays,2) < 7:
            m=re.search('[0-9][0-9][0-9][0-9][0-9][0-9]', userB.username)
            # filter for 6 integers in username. Teachers do not have 6 digits.
            if not m:  #if username doesn't contain six digits, run 
                userB.tags = userB.tags + newtags
                userB.update(tags=userB.tags)
                userB.update_role(role='org_publisher')
                print(userB.username + " updated to: " + userB.role)

print("complete.")